In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import requests
import json
from pprint import pprint
from dotenv import load_dotenv

In [48]:
# API Call
load_dotenv()
API_KEY = os.getenv("Yelp_API") # put your key here if you don't have a .env file in your main project folder.
headers = {'Authorization': 'Bearer %s' % API_KEY}
base_url = "https://api.yelp.com/v3/businesses/search"

In [88]:
# Test call
#search_limit_test = 10
#radius_test = 2000 #radius in meters
#term_test = "restaurants"
#location_test = "Atlanta" #use to test the json response
#search_params_test = {"location": location_test, "term": term_test, "limit": search_limit_test, "radius": radius_test,
                 #"sort_by": "rating"}
#response_test = requests.get(url = base_url, params = search_params_test, headers = headers).json()
#pprint(response_test)

In [4]:
stadium_data_df = pd.read_csv('Resources/stadium_coordinates.csv', dtype={
    "NFL Team": "string",
    "City": "string",
    "State": "string",
    "Stadium Name": "string",
    "Latitude": np.float64,
    "Longitude": np.float64,
    "Zip Code": "string"
})
stadium_data_df = stadium_data_df.dropna()
stadium_data_df

,NFL Team,City,State,Stadium Name,Latitude,Longitude,Zip Code
0,Arizona Cardinals,Glendale,Arizona,State Farm Stadium,33.528000,-112.263000,85305
1,Atlanta Falcons,Atlanta,Georgia,Mercedes-Benz Stadium,33.755556,-84.400000,30313
2,Baltimore Ravens,Baltimore,Maryland,M&T Bank Stadium,39.278056,-76.622778,21230
3,Buffalo Bills,Orchard Park,New York,Bills Stadium,42.774000,-78.787000,14127
4,Carolina Panthers,Charlotte,North Carolina,Bank of America Stadium,35.225833,-80.852778,28202
5,Chicago Bears,Chicago,Illinois,Soldier Field,41.862300,-87.616700,60605
6,Cincinnati Bengals,Cincinatti,Ohio,Paul Brown Stadium,39.095000,-84.516000,45202
7,Cleveland Browns,Cleveland,Ohio,FirstEnergy Stadium,41.506111,-81.699444,44114
8,Dallas Cowboys,Arlington,Texas,AT&T Stadium,32.747778,-97.092778,76011
9,Denver Broncos,Denver,Colorado,Empower Field at Mile High,39.741000,-105.019700,80204


In [188]:
# Make a list of coordinate tuples (lat, lng) for each stadium
stadium_coordinates = list(zip(stadium_data_df["Latitude"], stadium_data_df["Longitude"]))
#print(stadium_coordinates)

In [189]:
# Loop to search restaurants around the stadiums
search_limit = 20 #imit to 50 business names per query
radius = 3000 #radius in meters
term = "restaurants"
search_params = {"term": term, "limit": search_limit, "radius": radius, "sort_by": "rating"}
#search_params = {"term": term, "radius": radius, "sort_by": "rating"}
# lists to store the data retrieve from Yelp

business_city = []
business_lat = []
business_lng = []
business_price = []
business_rating = []
business_reviews = []
number_business = []
# Dictionary to create a dataframe from the api request
business_dict = {"Name": business_name, "City": business_city, "Price": business_price, 
                 "Rating": business_rating, "Number of reviews": business_reviews, 
                 "Latitude": business_lat, "Longitude": business_lng}
# list of coordinates to use to test the for loop.
coordinates_test = [(33.528, -112.26299999999999), (33.755556, -84.4)]

In [190]:
# Loop to retrieve the data based on (lat,lng)
for latitude, longitude in stadium_coordinates:
        
    search_params.update({"latitude": latitude, "longitude": longitude})
    response = requests.get(url = base_url, params = search_params, headers = headers).json()
    
    number_business.append(response['total'])
    
    #search_params.update({"limit": limite}) 
    
    #for i in range(10):
        #try:
            
            #business_price.append(response['businesses'][i]['price'])
            #business_rating.append(response['businesses'][i]['rating'])
            #business_reviews.append(response['businesses'][i]['review_count'])
            #business_city.append(response['businesses'][i]['location']['city'])
           
        #except:
            #business_price.append("NaN")
            
    

In [191]:
print(number_business)

[94, 707, 667, 51, 441, 1400, 458, 354, 210, 574, 531, 120, 180, 354, 175, 17, 756, 293, 293, 84, 681, 44, 946, 133, 133, 395, 380, 180, 1400, 269, 527, 94]


In [192]:
#print(business_rating)

In [193]:
#Add number of businesses to the data frame
stadium_data_df["Number of restaurants"]=number_business
stadium_data_df

,NFL Team,City,State,Stadium Name,Latitude,Longitude,Zip Code,Number of restaurants
0,Arizona Cardinals,Glendale,Arizona,State Farm Stadium,33.528000,-112.263000,85305,94
1,Atlanta Falcons,Atlanta,Georgia,Mercedes-Benz Stadium,33.755556,-84.400000,30313,707
2,Baltimore Ravens,Baltimore,Maryland,M&T Bank Stadium,39.278056,-76.622778,21230,667
3,Buffalo Bills,Orchard Park,New York,Bills Stadium,42.774000,-78.787000,14127,51
4,Carolina Panthers,Charlotte,North Carolina,Bank of America Stadium,35.225833,-80.852778,28202,441
5,Chicago Bears,Chicago,Illinois,Soldier Field,41.862300,-87.616700,60605,1400
6,Cincinnati Bengals,Cincinatti,Ohio,Paul Brown Stadium,39.095000,-84.516000,45202,458
7,Cleveland Browns,Cleveland,Ohio,FirstEnergy Stadium,41.506111,-81.699444,44114,354
8,Dallas Cowboys,Arlington,Texas,AT&T Stadium,32.747778,-97.092778,76011,210
9,Denver Broncos,Denver,Colorado,Empower Field at Mile High,39.741000,-105.019700,80204,574


In [164]:
# 15000 meters (10 miles) radius
#big_radius = 15000 #radius in meters
#search_params_big = {"term": term, "limit": search_limit, "radius": big_radius, "sort_by": "rating"}
#big_number_business = []
#for latitude, longitude in stadium_coordinates:
        
    #search_params_big.update({"latitude": latitude, "longitude": longitude})
    #response2 = requests.get(url = base_url, params = search_params_big, headers = headers).json()
    
    #big_number_business.append(response2["total"])

In [37]:
#print(big_number_business)

In [72]:
pprint(response)

{'businesses': [{'alias': 'alesha-donuts-glendale',
                 'categories': [{'alias': 'donuts', 'title': 'Donuts'},
                                {'alias': 'juicebars',
                                 'title': 'Juice Bars & Smoothies'},
                                {'alias': 'sandwiches', 'title': 'Sandwiches'}],
                 'coordinates': {'latitude': 33.53712, 'longitude': -112.23953},
                 'display_phone': '(623) 388-4613',
                 'distance': 2411.2311844542664,
                 'id': 'iucnfMFJlkL8sMYW9J0K8A',
                 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/XmtkfjEt5sg4v2BInj3ung/o.jpg',
                 'is_closed': False,
                 'location': {'address1': '8345 W Glendale Ave',
                              'address2': '',
                              'address3': None,
                              'city': 'Glendale',
                              'country': 'US',
                              'display_addres

In [144]:
print(business_rating)

[3.5, 4.5, 4.5, 3.0, 4.5, 4.5, 4.5, 4.5, 4.5, 4.5, 5.0, 3.5, 4.5, 4.5, 4.5, 4.5, 4.5, 4.5, 4.5, 4.0, 4.0, 4.5, 4.0, 4.0, 4.5, 4.0, 4.5]


In [145]:
print(business_reviews)

[443, 599, 216, 65, 427, 807, 1224, 253, 43, 607, 37, 90, 293, 134, 1022, 123, 123, 941, 1320, 106, 106, 139, 720, 2289, 7380, 496, 962]


In [167]:
#print(business_price)

In [168]:
#print(business_city)

In [183]:
#Test call
stadium_cities = stadium_data_df['City']
search_limit_test = 50
radius_test = 3000 #radius in meters
term_test = "restaurants"
#location_test = "Atlanta" #use to test the json response
search_params_test = {"location": location_test, "term": term_test, "limit": search_limit_test, "radius": radius_test,
                 "sort_by": "rating"}
business_price_1 = []
business_rating_1 = []
business_reviews_1 = []
number_business_1 = []
for city in stadium_cities:
    location_test = city
    response_test = requests.get(url = base_url, params = search_params_test, headers = headers).json()
    number_business_1.append(response_test['total'])
    try:
        for i in range(search_limit_test):
            business_price_1.append(response['businesses'][i]['price'])
            business_rating_1.append(response['businesses'][i]['rating'])
            business_reviews_1.append(response['businesses'][i]['review_count'])
    except:
            business_price.append("NaN")
#pprint(response_test)

In [194]:
pprint(response_test)

{'businesses': [{'alias': 'puranpoli-santa-clara',
                 'categories': [{'alias': 'vegetarian', 'title': 'Vegetarian'},
                                {'alias': 'indpak', 'title': 'Indian'}],
                 'coordinates': {'latitude': 37.376863,
                                 'longitude': -121.961358},
                 'display_phone': '(408) 352-5949',
                 'distance': 1863.7226648321366,
                 'id': 'DSKKwbvVmsrgUyKb53Aggg',
                 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/rDIPwFiO5BkSopwbQjpyiA/o.jpg',
                 'is_closed': False,
                 'location': {'address1': '3074B Scott Blvd',
                              'address2': None,
                              'address3': '',
                              'city': 'Santa Clara',
                              'country': 'US',
                              'display_address': ['3074B Scott Blvd',
                                                  'Santa Clara, C

In [195]:
len(business_price_1)

416

In [196]:
print(business_price_1)

['$$', '$$', '$$', '$', '$', '$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$', '$', '$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$', '$', '$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$', '$', '$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$', '$', '$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$', '$', '$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$', '$', '$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$', '$', '$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$', '$', '$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$', '$', '$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$', '$', '$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$', '$', '$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$', '$', '$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$', '